In [ ]:
from main import *
from hyperparameter import *
from transferlearning import *

#Set to False to also run hyperparameter tuning and transfer learning
JUSTBASELINE = True

# Preprocessing

## Set the paths based on your Drive directory

In [ ]:

data_dir ='data' # Change to the path to your directory
train_images_dir = os.path.join(data_dir, 'Training')
test_images_dir = os.path.join(data_dir, 'Testing')
print(f"Data directory: {data_dir}")
print(f"Training directory: {train_images_dir}")
print(f"Testing directory: {test_images_dir}")

## Preprocessing


### Don't touch this block

In [ ]:
# Load the training and test data
try:
    train_data, test_data = get_images(train_images_dir, test_images_dir)
    print("Data loading completed successfully.")
    print(f"Number of training samples: {train_data.samples}")
    print(f"Number of test samples: {test_data.samples}")
    print(f"Class names: {train_data.class_indices}")
except Exception as e:
    print(f"Error loading data: {e}")
    raise

# Data visualization

## Visualizing 15 random samples

In [ ]:
visualize_batch(train_data)

## Visualizing class distribution

In [ ]:
plot_class_distribution(train_data, test_data)

# Training, validation, and test set

In [ ]:
X_train, y_train, X_val, y_val = train_val_split(train_data)
X_test, y_test = test_splits(test_data)

# Baseline Model

In [ ]:
class_names_train = list(train_data.class_indices.keys())
class_names_test = list(test_data.class_indices.keys())

In [ ]:
base = baseline()
history = base.fit(X_train, y_train, batch_size =32, epochs = 10, validation_data = (X_val, y_val))

In [ ]:
models = [base]
metr, f1 = get_metrics(models, X_test, y_test, class_names_test)
display(metr, f1)

In [ ]:
summarize_metric(history, "accuracy")
summarize_metric(history, "loss")

roc_auc_val = plot_roc_curve(base, X_val, y_val, class_names_train,title ="Validation Set")
plot_confusion_matrix(base, X_val, y_val, class_names_train, normalize=False,title ="Validation Set")

roc_auc_val = plot_roc_curve(base, X_test, y_test, class_names_test, title = "Test Set")
plot_confusion_matrix(base, X_test, y_test, class_names_test, normalize=False, title = "Test Set")

# Hyper parameter tuning
dense units, filters, kernels, max pooling, activation, LR are all tuneable

In [ ]:
if JUSTBASELINE:
    print("Set JUSTBASELINE to False to run the following blocks")
    %

# change this project name for every run 
project_name = "baseline_hyperparameter_tuning1"
tuner_result = tune_hyperparameters(model_hyperparameter_tuning, project_name)
print_tuning_summary(tuner_result, project_name)

# Transfer learning

## VGG16

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
import transferlearning as tfl
resolution=32

vgg16m_model = tfl.transfer_learning(VGG16, resolution,X_train, y_train, X_val, y_val, epochs=20, batch_size=32, optimizer = "adam")

roc_auc_val = plot_roc_curve(vgg16m_model, X_val, y_val, class_names_train,title ="Validation Set")
plot_confusion_matrix(vgg16m_model, X_val, y_val, class_names_train, normalize=False,title ="Validation Set")

roc_auc_val = plot_roc_curve(vgg16m_model, X_test, y_test, class_names_test, title = "Test Set")
plot_confusion_matrix(vgg16m_model, X_test, y_test, class_names_test, normalize=False, title = "Test Set")

## Resnet50

In [ ]:
"""from tensorflow.keras.applications import ResNet50

resnet50_model = transfer_learning(ResNet50, resolution,X_train, y_train, X_val, y_val, epochs=10, batch_size=32, optimizer = "adam")

roc_auc_val = plot_roc_curve(resnet50_model, X_val, y_val, class_names_train,title ="Validation Set")
plot_confusion_matrix(resnet50_model, X_val, y_val, class_names_train, normalize=False,title ="Validation Set")

roc_auc_val = plot_roc_curve(resnet50_model, X_test, y_test, class_names_test, title = "Test Set")
plot_confusion_matrix(resnet50_model, X_test, y_test, class_names_test, normalize=False, title = "Test Set")"""

## Densenet121

In [ ]:
"""from tensorflow.keras.applications import DenseNet121

densenet121_model = transfer_learning(DenseNet121, resolution,X_train, y_train, X_val, y_val, epochs=10, batch_size=32, optimizer = "adam")

roc_auc_val = plot_roc_curve(densenet121_model, X_val, y_val, class_names_train,title ="Validation Set")
plot_confusion_matrix(densenet121_model, X_val, y_val, class_names_train, normalize=False,title ="Validation Set")

roc_auc_val = plot_roc_curve(densenet121_model, X_test, y_test, class_names_test, title = "Test Set")
plot_confusion_matrix(densenet121_model, X_test, y_test, class_names_test, normalize=False, title = "Test Set")"""

## This is a model suggested in a medium article for mnist that I tried for our dataset and it works quite well: https://medium.com/@BrendanArtley/mnist-keras-simple-cnn-99-6-731b624aee7f. Getting 84%

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout, BatchNormalization, Activation, Input, GlobalAveragePooling2D, LSTM, Embedding, GRU, Conv1D, Conv3D
def model_mnist():
    model = Sequential()
    
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same',
                     input_shape=(30,30,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid' ))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), padding='valid', strides=2))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='softmax'))
    optimizer = Adam(0.001, beta_1=0.9, beta_2=0.999 )
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model

mnist_model = model_mnist()
reduce_lr = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
history = mnist_model.fit(X_train, y_train, batch_size =32, epochs = 30, validation_data = (X_val, y_val), callbacks = [reduce_lr])

## Pim's model. Getting 80%

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential,  layers, Input, optimizers

def hyperparam(): 
    model = Sequential()
    model.add(Input(shape = (30, 30, 1)))
    
    model.add(layers.Conv2D(128, (3, 3), activation="relu", padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3, 3), activation="relu", padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2), strides=2))
    
    model.add(layers.Conv2D(32, (3, 3), activation="relu", padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(16, (3, 3), activation="relu", padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2, 2))

    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation= 'relu'))
    model.add(layers.Dropout(rate = 0.5))
    model.add(layers.Dense(32, activation = "relu"))
    model.add(layers.Dropout(rate = 0.25))
    model.add(layers.Dense(16, activation = 'relu'))
    model.add(layers.Dense(4, activation = "softmax"))

    model.compile(optimizer = optimizers.Adam(learning_rate= 0.001),
                    loss = "categorical_crossentropy",
                    metrics = ["accuracy"])
    return model

model = hyperparam()
print(model.summary())

model.fit(X_train, y_train, batch_size =32, epochs = 30, validation_data = (X_val, y_val), verbose = 1)

## Lenet5

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential,  layers, Input, optimizers

def lenet5(): 
    model = Sequential()
    model.add(Input(shape = (30, 30, 1)))
    
    model.add(layers.Conv2D(6, (5, 5), strides=1, activation="tanh", padding='same'))
    model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=2, padding='valid'))
    
    model.add(layers.Conv2D(16, (5, 5), strides=1, activation="tanh", padding='valid'))
    model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=2, padding='valid'))
    
    model.add(layers.Conv2D(filters=120, kernel_size=(5, 5), strides=1, padding='valid', activation='tanh'))
    
    model.add(Flatten())
    
    model.add(layers.Dense(units=84, activation='tanh'))
    
    model.add(layers.Dense(units=4, activation='softmax'))

    model.compile(optimizer = optimizers.Adam(learning_rate= 0.001),
                    loss = "categorical_crossentropy",
                    metrics = ["accuracy"])
    return model

model = lenet5()
print(model.summary())

model.fit(X_train, y_train, batch_size =16, epochs = 30, validation_data = (X_val, y_val), verbose = 1)